In [1]:
import collections
import copy

def ld():
    return [int(x) for x in open("21.txt").read().split(',')]


class m:
    idgen=0
    
    #get op and modes of following 3 addresses
    def ix(s,x):
        assert(x>=0)
        x="%05d"%(x)
        op=int(x[3:])
        m3,m2,m1=[int(c) for c in x[:3]]
        return op,m1,m2,m3


    def gv(s,p,ip,m,ix):
        assert(ix in [1,2,3])
        if m==0:#reg
            return p[p[ip+ix]]
        elif m==1:#imm
            return p[ip+ix] 
        elif m==2:#reg+rb
            return p[p[ip+ix]+s.rb]
        assert(0)
        
            
        

    
    def __init__(s,p):
        #load to mem
        s.p=collections.defaultdict(int)
        for i,v in enumerate(p):
            s.p[i]=v
            
        #possible data source
        s.supplier=None
        
        #input queue
        s.i=[]
        
        #id
        s.id=m.idgen
        m.idgen+=1
        
        #iterator for execution
        #s.it=s.iterator()
        s.ip=0
        
        #print("M",s.id,"created")
        
        #relative base
        s.rb=0
        
    
    def sendInput(s, i):
        s.i.append(i)
        
    
    def getData(s):
        ip=s.ip
        
        while 1:
            op,m1,m2,m3=s.ix(s.p[ip])
            
            if op==1:#add
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)+s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==2:#mul
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)*s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==3:#in
                assert(m1 in [0,2])
                r=None
                if len(s.i)>0:
                    r=s.i[0]
                    s.i=s.i[1:]
                else:
                    if s.supplier:
                        r=s.supplier.getData()
                    else:
                        print("id",s.id, "returning zero")
                        #assert(0)
                        r=0
                tix=s.p[ip+1]
                if m1==2:
                    tix+=s.rb
                s.p[tix]=r
                ip+=2
            elif op==4:#out
                
                r=s.gv(s.p,ip,m1,1)
                #print(s.id,"spits out",r)
                s.ip=ip+2
                return r
                #print("--id:",s.id,"onwards")
                
                
            elif op==5:#jump-if-true
                if s.gv(s.p,ip,m1,1):
                    ip=s.gv(s.p,ip,m2,2)
                else:
                    ip+=3
            elif op==6:#jump-if-false
                if not s.gv(s.p,ip,m1,1):
                    ip=s.gv(s.p,ip,m2,2)
                else:
                    ip+=3
            elif op==7:#less-than
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)<s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==8:#equals
                assert(m3 in [0,2])
                tix=s.p[ip+3]
                if m3==2:
                    tix+=s.rb
                s.p[tix]=s.gv(s.p,ip,m1,1)==s.gv(s.p,ip,m2,2)
                ip+=4
            elif op==9:#adjust relative base
                s.rb+=s.gv(s.p,ip,m1,1)
                ip+=2
            elif op in [99]:#end
                #print("fin", s.id)
                break
            else:
                print("id", s.id, "op",op,"c",c)
                assert(0)
            #print(op)
            
        
        return -1
    
#ld()    
    
    

In [2]:
from PIL import Image
from IPython import display
import numpy as np
import cv2
import random

def pretty(o):
    o=o.split('\n')
    h=len(o)
    w=len(o[0])
    i=np.zeros([h,w,3],dtype=np.uint8)
    c={}
    c[' ']=[20,20,20]
    c['@']=[255,0,255]  
    c['#']=[255,255,255]  
    c['.']=[0,0,255]
    for y,l in enumerate(o):
        for x,v in enumerate(l):
            i[y,x,:]=c[v]
            if v=='#' and x%2==0:
                i[y,x,:]-=55
    F=4
    i = cv2.resize(i, fx=F, fy=F, dsize=(0, 0), interpolation=cv2.INTER_NEAREST)
    display.clear_output(wait=True)
    display.display(Image.fromarray(i))

    
            

In [3]:
import copy
import time

def test(code,display=1):

    p=ld()
    M=m(p)
    v=0
    for l in code:
        for c in l:
            M.sendInput(ord(c))
        M.sendInput(ord('\n'))
    steps=0
    o=""
    while 1: #steps < 100:
        v=M.getData()
        if v>= 0:
            if v > 255:
                return v
            o+=chr(v)
            #print(o)
            if o[-2:]=="\n\n":
                #print(o)
                if "....." in o:
                    if display:
                        pretty(o)
                        time.sleep(0.6)
                        #print(o)
                elif not "Didn't" in o:
                    if not "Input" in o:
                        if not "Walking" in o:
                            if not "Running" in o:
                                print(o)
                                return -1
                o=""
        else:
            break
        
        steps+=1
    #pretty(o)
    return 0



     
#J T
#ABCD
#AND X Y
#OR  X Y
#NOT X Y
#ground == true
    
def p1():
    code=[]
    
    
    
    code.append("NOT J J")
    code.append("AND A J")
    code.append("AND B J")
    code.append("AND C J")
    code.append("NOT J J")
    code.append("AND D J")
    #code.append("AND T J")
    
    code.append("WALK")
    
    v=test(code)
    #print(v)
    return v
    
print("part1:", p1(),19359533)


part1: 19359533 19359533


In [4]:
def p2():
    #@   x   x
    #@   xx   x
    # abcdefghi
    code=[]
    #j=(!a|!b|!c)&((d&h)|(d&e&i))
    
    #expression above was correct but
    #missed abiloty for further planning.
    
    #adding a rule saying if hole 
    #immediately just jump
    
    #j=!a|((!b|!c)&((d&h)|(d&e&i)))
    
    #(d&h) in T
    code.append("NOT T T")
    code.append("AND D T")
    code.append("AND H T")
    #(d&e&i) in J
    code.append("NOT J J")
    code.append("AND D J")
    code.append("AND E J")
    code.append("AND I J")
    #(d&h)|(d&e&i) in T
    code.append("OR J T")
    #holes in J (use d to start)
    code.append("OR T J")
    #code.append("AND A J") # removed as !a trigger jump
    code.append("AND B J")
    code.append("AND C J")
    code.append("NOT J J")
    #let !a trigger jump
    code.append("AND T J")
    code.append("NOT A T")
    code.append("OR T J")
    
    
    code.append("RUN")
    
    v=test(code)
    
    return v
    
print("part2:", p2(), 1140310551)


part2: 1140310551 1140310551
